# 特征工程

## 对合并后的集进行缺失值填补

In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import  DecisionTreeClassifier,export_graphviz
from xgboost import plot_importance
from matplotlib import pyplot as plt
from collections import Counter
from imblearn.over_sampling import RandomOverSampler

In [3]:
# 缺失值填充
dt = pd.read_csv("dt.csv")
pax_fcny = dt['pax_fcny'].mode()
dt.loc[dt['pax_fcny'].isnull(),'pax_fcny']= pax_fcny[0]
pax_tax = dt['pax_tax'].mode()
dt.loc[dt['pax_tax'].isnull(),'pax_tax']= pax_tax[0]
seg_cabin=dt['seg_cabin'].mode()
dt.loc[dt['seg_cabin'].isnull(),'seg_cabin']=seg_cabin[0]

## 异常值处理

In [4]:
labels = dt['emd_lable2']
X = dt
X1 = X.drop(['emd_lable2','pax_name'],axis=1)

## 归一化

In [19]:
mm= MinMaxScaler()
# 对测试集和训练集的特征值进行归一化
X2 = mm.fit_transform(X1)
X3=  pd.DataFrame(X2,columns=X1.columns)
# X3.to_csv('X1.csv')

## 箱线图四分法异常值替换

In [6]:
def box_fill(col):
    que = col.quantile(0.75)-col.quantile(0.25)#数据四分之三分位值与四分之一分位值的差
    up_th = col.quantile(0.75) + 1.5*que#上界
    low_th = col.quantile(0.25) - 1.5*que#下界
    def box_change(x):#定义转换函数，如果数字大于上界则用上界值填充，小于下界则用下界值填充。
        if x > up_th:
            return up_th
        elif x < low_th:
            return low_th
        else:
            return x
    return col.map(box_change)
for index in X3:
    X3[index] = box_fill(X3[index])


In [7]:
X4 = pd.DataFrame(X3,columns=X1.columns)
X4.insert(0,'emd_lable2',labels.values)
X4

C:\Users\yaokexin\AppData\Local\Temp\ipykernel_5172\1425414722.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X4.insert(0,'emd_lable2',labels.values)


,emd_lable2,Unnamed: 0,seg_route_to,seg_flight,seg_cabin,pax_fcny,pax_tax,cabin_hf_cnt_m3,cabin_hf_cnt_m6,cabin_hf_cnt_y1,...,pit_pay_cnt_y2,pit_pay_cnt_y3,pit_income_avg_amt_m3,pit_income_avg_amt_m6,pit_income_avg_amt_y1,pit_income_avg_amt_y2,pit_income_avg_amt_y3,pit_pay_avg_amt_m3,pit_pay_avg_amt_m6,pit_pay_avg_amt_y1
0,0.0,0.000000,0.0,0.3750,0.8,0.000596,0.000809,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000043,0.0,0.3750,0.8,0.000392,0.000344,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000085,0.5,0.5625,0.8,0.000057,0.000417,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000128,0.5,0.5625,0.8,0.000319,0.000629,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.000171,0.0,0.6250,0.8,0.000158,0.000393,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30198,0.0,0.288763,0.5,0.5000,0.8,0.000596,0.000705,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30199,0.0,0.288805,0.5,0.5000,0.8,0.000596,0.001018,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30200,0.0,0.288848,0.0,0.3750,0.8,0.000076,0.000381,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30201,0.0,0.288891,0.0,0.6250,0.8,0.000090,0.000391,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 拆分训练集以及验证集

In [8]:
df1=X4.loc[:23431]
df1

,emd_lable2,Unnamed: 0,seg_route_to,seg_flight,seg_cabin,pax_fcny,pax_tax,cabin_hf_cnt_m3,cabin_hf_cnt_m6,cabin_hf_cnt_y1,...,pit_pay_cnt_y2,pit_pay_cnt_y3,pit_income_avg_amt_m3,pit_income_avg_amt_m6,pit_income_avg_amt_y1,pit_income_avg_amt_y2,pit_income_avg_amt_y3,pit_pay_avg_amt_m3,pit_pay_avg_amt_m6,pit_pay_avg_amt_y1
0,0.0,0.000000,0.0,0.3750,0.8,0.000596,0.000809,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000043,0.0,0.3750,0.8,0.000392,0.000344,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000085,0.5,0.5625,0.8,0.000057,0.000417,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000128,0.5,0.5625,0.8,0.000319,0.000629,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.000171,0.0,0.6250,0.8,0.000158,0.000393,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23427,0.0,0.999829,1.0,0.4375,0.8,0.000189,0.000674,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23428,0.0,0.999872,1.0,0.4375,0.8,0.000520,0.000433,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23429,0.0,0.999915,1.0,0.4375,0.8,0.000520,0.000433,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23430,0.0,0.999957,1.0,0.4375,0.8,0.000520,0.000433,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
ts1=X4.loc[23432:]
ts1=ts1.reset_index(drop = True)
ts1.to_csv("D:/XGboost/ts1.csv")
ts1

,emd_lable2,Unnamed: 0,seg_route_to,seg_flight,seg_cabin,pax_fcny,pax_tax,cabin_hf_cnt_m3,cabin_hf_cnt_m6,cabin_hf_cnt_y1,...,pit_pay_cnt_y2,pit_pay_cnt_y3,pit_income_avg_amt_m3,pit_income_avg_amt_m6,pit_income_avg_amt_y1,pit_income_avg_amt_y2,pit_income_avg_amt_y3,pit_pay_avg_amt_m3,pit_pay_avg_amt_m6,pit_pay_avg_amt_y1
0,0.0,0.000000,0.0,0.6250,0.8,0.000079,0.000315,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000043,0.5,0.5000,0.8,0.000060,0.000346,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000085,0.0,0.6250,0.8,0.000112,0.000569,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000128,0.5,0.5625,0.8,0.000094,0.000791,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000171,0.0,0.3750,0.8,0.000117,0.000387,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6766,0.0,0.288763,0.5,0.5000,0.8,0.000596,0.000705,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6767,0.0,0.288805,0.5,0.5000,0.8,0.000596,0.001018,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6768,0.0,0.288848,0.0,0.3750,0.8,0.000076,0.000381,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6769,0.0,0.288891,0.0,0.6250,0.8,0.000090,0.000391,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 过采样处理

In [10]:
y = df1['emd_lable2']
x = df1
x = x.drop(['emd_lable2'],axis=1)

In [11]:
ros = RandomOverSampler(random_state=0)#随机过采样
X_resampled, y_resampled = ros.fit_resample(x, y)#使用原始数据的特征变量和目标变量生成过采样数据集
sorted(Counter(y_resampled).items())

[(0.0, 21957), (1.0, 21957)]

In [13]:
gc=X_resampled
gc['emd_lable2']=y_resampled
gc.to_csv("D:/XGboost/gc.csv")
gc

,Unnamed: 0,seg_route_to,seg_flight,seg_cabin,pax_fcny,pax_tax,cabin_hf_cnt_m3,cabin_hf_cnt_m6,cabin_hf_cnt_y1,cabin_hf_cnt_y2,...,pit_pay_cnt_y3,pit_income_avg_amt_m3,pit_income_avg_amt_m6,pit_income_avg_amt_y1,pit_income_avg_amt_y2,pit_income_avg_amt_y3,pit_pay_avg_amt_m3,pit_pay_avg_amt_m6,pit_pay_avg_amt_y1,emd_lable2
0,0.000000,0.0,0.3750,0.8,0.000596,0.000809,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000043,0.0,0.3750,0.8,0.000392,0.000344,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000085,0.5,0.5625,0.8,0.000057,0.000417,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000128,0.5,0.5625,0.8,0.000319,0.000629,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000171,0.0,0.6250,0.8,0.000158,0.000393,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43909,0.013913,0.0,0.6250,0.8,0.000098,0.000415,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
43910,0.631428,0.0,0.6250,0.8,0.000166,0.000628,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
43911,0.639623,0.5,0.5000,0.8,0.000040,0.000365,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
43912,0.313986,0.0,0.3750,0.8,0.000178,0.000629,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
